In [1]:
import scipy

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.api import anova_lm

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Load data
df = pd.read_csv('F:/Desktop/甜的线归.csv',encoding='gbk')
df.head(10)

,AccountActivity3,Balance1,Payment1,PrevBalance1,HistoricalAccountActivity2,HistoricalAccountActivity3,HistoricalAccountActivity4,HistoricalAccountActivity5,HistoricalAccountDetail1,HistoricalAccountDetail2,HistoricalAccountDetail3,HistoricalAccountDetail4,HistoricalAccountStatus1,HistoricalBalance1,Target
0,14.95,606.12,30.00,566.17,523.21,92.76,15,386.15,456.57,85,99,0,30,594,0
1,0.00,297.97,75.37,327.97,436.82,25.20,9,200.55,326.47,113,103,0,15,395,0
2,0.00,0.00,200.61,0.00,75.00,0.99,4,875.61,339.21,100,21,0,10,600,0
3,88.82,323.60,100.00,285.10,569.77,26.96,50,574.54,308.76,109,85,0,16,382,0
4,226.94,271.76,30.00,301.76,0.00,0.00,5,352.06,124.65,60,0,0,3,302,0
5,0.00,300.81,0.00,368.81,261.26,68.31,13,255.17,306.08,80,67,0,31,400,0
6,115.78,585.28,60.00,585.28,314.00,12.43,6,525.30,401.08,98,99,0,14,585,0
7,31.33,920.06,50.00,956.79,1202.91,192.39,8,243.25,944.30,99,112,0,35,988,0
8,285.25,421.97,228.88,287.78,1280.01,47.38,118,2748.38,221.47,18,17,0,35,858,0
9,0.00,733.14,50.00,613.00,808.83,153.51,5,329.95,531.33,76,102,0,35,649,0


In [6]:
#计算相关系数
cor_matrix = df.corr(method='pearson')  # 使用皮尔逊系数计算列与列的相关性
# cor_matrix = df.corr(method='kendall')
# cor_matrix = df.corr(method='spearman')

print(cor_matrix)#结果表现为全低相关，意料之中

                            AccountActivity3  Balance1  Payment1  \
AccountActivity3                    1.000000  0.132273  0.193781   
Balance1                            0.132273  1.000000  0.013887   
Payment1                            0.193781  0.013887  1.000000   
PrevBalance1                        0.133771  0.968929 -0.012192   
HistoricalAccountActivity2          0.165303  0.328947  0.297641   
HistoricalAccountActivity3         -0.040454  0.671437  0.087664   
HistoricalAccountActivity4          0.243184  0.150515  0.220232   
HistoricalAccountActivity5          0.414044  0.231161  0.348846   
HistoricalAccountDetail1           -0.021841  0.901343  0.124195   
HistoricalAccountDetail2           -0.087376  0.263448 -0.068494   
HistoricalAccountDetail3           -0.137408  0.195038  0.024425   
HistoricalAccountDetail4           -0.013505  0.021292  0.004484   
HistoricalAccountStatus1           -0.067337  0.365091  0.081377   
HistoricalBalance1                  0.116550  0.

In [7]:
#-----------前进法----------#
var = set(df.columns)#w为了达到任意加入的效果先使用集合
var.remove('Target')
variate = list(var)
selected = []
current_score, best_new_score = float('inf'),0.0 
while variate:
    aic_with_variate=[]
    for candidate in variate:  #逐个遍历自变量
            formula="{}~{}+1".format('Target',"+".join(selected + [candidate]))  #将自变量名连接起来
            aic=smf.ols(formula=formula,data=df).fit().aic  #利用ols训练模型得出aic值
            aic_with_variate.append((aic,candidate))  #将第每一次的aic值放进空列表
    aic_with_variate.sort(reverse=True)  #降序排序aic值
    #print(aic_with_variate) #此结果为variate列表中依次加入元素所得到的aic值及加入元素的名称的排序结果
    best_new_score,best_candidate=aic_with_variate.pop(-1)  #最好的aic值以及最好的自变量等于列表的最后一个值和自变量
    if (current_score>best_new_score):  #如果目前的aic值大于最好的aic值
        variate.remove(best_candidate)  #移除加进来的变量名，即第二次循环时，不考虑此自变量了
        selected.append(best_candidate)  #将此对模型有益的自变量（第一次循环为单个变量，后续为组合）挑选出来，一直参与循环
        print("目前的最优自变量组合：",end = '')
        print(selected)
        current_score=best_new_score  #最新的分数等于最好的分数
        print("最小的AIC值为： {}!".format(current_score))  #输出最小的aic值
        print('')
    else:
        print("自变量选择完毕!")
        break
formula="{}~{}".format('Target',"+".join(selected))  #最终的模型式子
print("最优的回归方程模型： {}".format(formula))
model=smf.ols(formula=formula,data=df).fit()

目前的最优自变量组合：['HistoricalAccountDetail2']
最小的AIC值为： 386708.12363682257!

目前的最优自变量组合：['HistoricalAccountDetail2', 'HistoricalAccountStatus1']
最小的AIC值为： 377950.98391987465!

目前的最优自变量组合：['HistoricalAccountDetail2', 'HistoricalAccountStatus1', 'Payment1']
最小的AIC值为： 371086.9057698398!

目前的最优自变量组合：['HistoricalAccountDetail2', 'HistoricalAccountStatus1', 'Payment1', 'HistoricalAccountActivity5']
最小的AIC值为： 369707.58626557246!

目前的最优自变量组合：['HistoricalAccountDetail2', 'HistoricalAccountStatus1', 'Payment1', 'HistoricalAccountActivity5', 'Balance1']
最小的AIC值为： 368011.66868787626!

目前的最优自变量组合：['HistoricalAccountDetail2', 'HistoricalAccountStatus1', 'Payment1', 'HistoricalAccountActivity5', 'Balance1', 'HistoricalAccountDetail1']
最小的AIC值为： 364553.9300318371!

目前的最优自变量组合：['HistoricalAccountDetail2', 'HistoricalAccountStatus1', 'Payment1', 'HistoricalAccountActivity5', 'Balance1', 'HistoricalAccountDetail1', 'AccountActivity3']
最小的AIC值为： 362427.1326230361!

目前的最优自变量组合：['HistoricalAccountDetail2', 'Histo

In [8]:
para = model.params
print(model.params)

Intercept                     0.194588
HistoricalAccountDetail2      0.005180
HistoricalAccountStatus1     -0.006597
Payment1                     -0.000233
HistoricalAccountActivity5   -0.000098
Balance1                      0.000407
HistoricalAccountDetail1     -0.000441
AccountActivity3             -0.000391
HistoricalAccountDetail4      0.048276
HistoricalAccountActivity2    0.000018
HistoricalAccountActivity4   -0.000887
PrevBalance1                  0.000059
HistoricalAccountActivity3   -0.000082
HistoricalAccountDetail3     -0.000128
dtype: float64


In [9]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 Target   R-squared:                       0.168
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     4453.
Date:                Mon, 30 Jan 2023   Prob (F-statistic):               0.00
Time:                        16:57:35   Log-Likelihood:            -1.8094e+05
No. Observations:              285827   AIC:                         3.619e+05
Df Residuals:                  285813   BIC:                         3.621e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [10]:
print(model.pvalues)

Intercept                      0.000000e+00
HistoricalAccountDetail2       0.000000e+00
HistoricalAccountStatus1       0.000000e+00
Payment1                      4.093036e-131
HistoricalAccountActivity5    2.468327e-269
Balance1                       0.000000e+00
HistoricalAccountDetail1       0.000000e+00
AccountActivity3               0.000000e+00
HistoricalAccountDetail4       2.569518e-43
HistoricalAccountActivity2     2.000069e-32
HistoricalAccountActivity4     3.143262e-26
PrevBalance1                   3.962759e-09
HistoricalAccountActivity3     5.148882e-06
HistoricalAccountDetail3       9.878046e-05
dtype: float64


In [55]:
data = df[['HistoricalAccountDetail2','HistoricalAccountStatus1','Payment1','HistoricalAccountActivity5','Balance1','HistoricalAccountDetail1','AccountActivity3','HistoricalAccountDetail4','HistoricalAccountActivity2','HistoricalAccountActivity4','PrevBalance1','HistoricalAccountActivity3','HistoricalAccountDetail3']]
r = model.predict(data)
len(r)

285827

In [56]:
def f(x,y):
    x = abs(x)
    if x>=y:
        return 1
    else:
        return 0
result = pd.DataFrame(map(lambda x:f(x,0.5),r))#寻找最优界限
result.head(10)

,0
0,0
1,1
2,0
3,1
4,0
5,0
6,1
7,0
8,0
9,0


In [57]:
a = 0
for i in range(len(t_res)):
    temp = abs(t_res[i]-result[0][i])
    if temp==1:
        a+=1
    else:
        continue
res = a/len(t_res)#错误率
res

0.32750929758210384